In [1]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# read from melodyData.txt
with open('generated/melodyDataMedium.txt', 'r') as f:
    parts = eval(f.read())
    print ("Number of parts:" + str(len(parts)))

# one-hot encoding

encodings = {(-1, 0): 0}
encodingIndex = 1
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1
print(encodings)

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])

Number of parts:106
{(-1, 0): 0, (15, '0.25'): 1, (0, '0.25'): 2, (16, '0.25'): 3, (14, '0.25'): 4, (13, '0.25'): 5, (13, '0.50'): 6, (11, '0.50'): 7, (10, '0.25'): 8, (8, '0.25'): 9, (11, '0.25'): 10, (8, '0.50'): 11, (0, '1.00'): 12, (0, '0.50'): 13, (23, '0.25'): 14, (20, '0.25'): 15, (19, '0.25'): 16, (22, '0.25'): 17, (15, '1.00'): 18, (0, '0.75'): 19, (20, '2.00'): 20, (0, '2.75'): 21, (27, '0.25'): 22, (28, '0.25'): 23, (26, '0.25'): 24, (25, '0.25'): 25, (25, '0.50'): 26, (23, '0.50'): 27, (27, '0.50'): 28, (8, '0.75'): 29, (20, '0.50'): 30, (18, '1.50'): 31, (16, '0.50'): 32, (15, '0.75'): 33, (0, '3.00'): 34, (21, '0.25'): 35, (26, '0.50'): 36, (3, '2.00'): 37, (5, '1.50'): 38, (5, '0.25'): 39, (3, '0.75'): 40, (3, '0.25'): 41, (2, '0.50'): 42, (3, '1.50'): 43, (0, '3.75'): 44, (15, '0.50'): 45, (18, '0.50'): 46, (12, '0.50'): 47, (4, '0.50'): 48, (9, '0.50'): 49, (18, '0.25'): 50, (6, '0.50'): 51, (18, '0.75'): 52, (0, '1.50'): 53, (9, '0.25'): 54, (6, '0.25'): 55, (2, '0.25

In [3]:
# sequence_length = 10

# X = []
# Y = []
# # given data_encoded, generate training data by looping 
# for i in range(len(data_encoded[:100])):
#     for j in range(len(data_encoded[i]) - sequence_length):
#         X.append(data_encoded[i][j:j + sequence_length])
#         Y.append(data_encoded[i][j + sequence_length])

# X_onehot = []
# for seq in X:
#     onehot = np.zeros((len(seq), len(encodings)))
#     for note_index in range(len(seq)):
#         onehot[note_index][seq[note_index]] = 1
#     X_onehot.append(onehot)
# X = X_onehot


# # split this into training and testing sets 
# # we will intentionally overfit, GET RID OF THIS LATER
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# #print (X_train) # each element of the list is a 2d array

In [4]:
# sequence_length = 10

# # X_train should contain sequences from parts 1-100
# # X_test should contain the answers to said sequences
# # Y_train should contains sequences from parts 201-210
# # Y_test should contain the answers to said sequences

# def generate_data(data_encoded, encodings, sequence_length=10):

#     X = []
#     Y = []
#     for i in range(len(data_encoded)):
#         for j in range(len(data_encoded[i]) - sequence_length):
#             X.append(data_encoded[i][j:j + sequence_length])
#             #print (data_encoded[i][j + sequence_length])
#             Y.append(data_encoded[i][j + sequence_length])
    
#     X_onehot = []

#     """
# X_onehot = []
# for seq in X:
#     onehot = np.zeros((len(seq), len(encodings)))
#     for note_index in range(len(seq)):
#         onehot[note_index][seq[note_index]] = 1
#     X_onehot.append(onehot)
# X = X_onehot
#     """

#     for seq in X:
#         onehot = np.zeros((len(seq), len(encodings)))
#         for note_index in range(len(seq)):
#             onehot[note_index][seq[note_index]] = 1
#         X_onehot.append(onehot)
#     X = X_onehot

#     return X, Y

# X_train, Y_train = generate_data(data_encoded[:100], encodings)
# X_test, Y_test = generate_data(data_encoded[200:205], encodings)

# #print (X_train[0])

In [5]:
# X_train should contain sequences from parts 1-100
# X_test should contain the answers to said sequences
# Y_train should contains sequences from parts 201-210
# Y_test should contain the answers to said sequences

start_token = np.zeros(len(encodings))
start_token[0] = 1
def generate_data(data_encoded, encodings):
    X = []
    Y = []
    for seq in data_encoded:
        onehot = np.zeros((len(seq) + 1, len(encodings)))
        # set start token
        onehot[0][0] = 1
        for note_index in range(len(seq)):
            onehot[note_index + 1][seq[note_index]] = 1
        X.append(onehot[:-1])
        Y.append(onehot[1:])
    return X, Y

num_tests = 5
X_train, Y_train = generate_data(data_encoded[:-num_tests], encodings)
X_test, Y_test = generate_data(data_encoded[-num_tests:], encodings)

print(X_train[0].shape)
print(len(Y_train[0]))

(58, 330)
58


In [11]:
#split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
# T    tf.keras.layers.Input(shape=(sequence_length, 1)), X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# TODO: make the model itself
# benchmark: dense is 80% accuracy
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(batch_shape=(1, None, len(encodings))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, stateful=True)),
    # tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def reset_model():
    global model
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()

def train_sequence(model, X_one, Y_one):
    X_one = np.array([X_one])
    Y_one = np.array([Y_one])
    model.fit(X_one, Y_one, batch_size=1, epochs=1, verbose=1)
    reset_model()

num_epochs = 20
for t in range(num_epochs):
    for i in range(len(X_train)):
        train_sequence(model, X_train[i], Y_train[i])
    print("Finished", t + 1, "epochs")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_1 (Bidirectional) │ (1, None, 256)         │       470,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (1, None, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (1, None, 128)         │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (1, None, 330)         │        42,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 561,994 (2.14 MB)

 Trainable params: 561,994 (2.14 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 5.7913
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0364 - loss: 5.7823
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.2609 - loss: 5.7752
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2593 - loss: 5.7797
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.3261 - loss: 5.7391
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.3750 - loss: 5.6706
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.3333 - loss: 5.7026
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.3282 - loss: 5.6545
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2754 - loss: 5.5880
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.0000e+00 - loss: 5.8118
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.0000e+00 - loss: 5.7755
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.0000e+00 - loss: 5.7559
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0000e+00 - loss: 5.6798
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accura

In [7]:
# X_moretest = np.append(X_train[0][:9], [X_train[0][3]], axis=0)
# # X_moretest = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
# # X_moretest = X_train[0]
# print(X_moretest)
# pred = model.predict(np.array([X_moretest]))[0]
# print(pred.shape)
# for i in range(pred.shape[0]):
#     print(np.argmax(pred[i]))

In [8]:
loss, accuracy = model.evaluate(np.array(X_test), np.array(Y_test), verbose=1)
print(f'Loss: {loss}, Accuracy: {accuracy}')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [372]:
# import copy
# seq = copy.deepcopy(X_test[0])

# def trim(seq):
#     for i in range(len(seq)):
#         if np.all(seq[i] == -1):
#             return seq[:i]
#     return seq[:]
# def pad(seq):
#     return np.pad(seq, ((0, max_length - len(seq)), (0, 0)), 'constant', constant_values=-1)

# total_seq = trim(seq)
# for i in range(10):
#     pred = model.predict(np.array([seq]))
#     pred = np.argmax(pred)
#     one_hot = np.zeros((len(encodings)))
#     one_hot[pred] = 1
#     total_seq = np.append(total_seq, [one_hot], axis=0)
#     seq = pad(total_seq[-max_length:])
#     seq[0] = start_token
#     print(pred)

In [12]:
import copy
# given a sequence, generate the next note (up to ten times)
seq = copy.deepcopy(X_test[4])
total_seq = seq[:]
for i in range(10):
    pred = model.predict(np.array([seq]))[0][-1]
    print(np.max(pred))
    pred = np.argmax(pred)
    # append to sequence the one-hot encoding of pred
    one_hot = np.zeros((len(encodings)))
    one_hot[pred] = 1
    total_seq = np.append(total_seq, [one_hot], axis=0)
    # set sequence to be the last ten values of total_seq
    seq = total_seq

    #total_seq.append(pred)
    # total seq is a numpy array, can't just append things to it

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}
    

for note in total_seq:
    pitch, dur = decodings[np.argmax(note)]
    if (pitch == -1):
        print("Start...")
    elif (pitch == 0):
        print("Rest for", dur)
    else:
        print(f"{mapping[(pitch + 11) % 12]}{(decodings[np.argmax(note)][0] - 1) // 12 + 4} for {dur}")

# convert numbers to musical note letters
# 60 = C

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
0.24538931
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
0.96685255
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
0.9250226
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.9135511
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
0.85998803
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.7579042
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.5930738
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
0.4819627
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
0.9702105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.95843077
Start...
Rest for 2.00
G4 for 2.00
G#4 for 2.00
F4 for 2.00
G4 for 3.00
G4 for 1.00
C4 for 4.00
Rest for 2.00
G4 for 2.00
D#4 for 2.00
F4 for 2.00
D4 for 3.00
D4 for 1.00
C4 for 4.00
Rest for 8.00
Rest for 6.00
C5 for 2.00
C5 for 1.00
C5 for 1.00
B4 for 2.00
C5 for 2.00
D#5 for 2.00
D5 for 4.00
C5 for 2.00
F4 for 2.00
C5 for 6.00
C5 for 2.00
A#4 for 2.00
A#4 for 2.00
G#4 for 4.00
G#4 for 2.00
G#4 for 2.00
G4 for 4.00
G4 for 4.00
Rest for 2.00
C4 for 2.00
C4 for 2.00
C4 for 2.00
D#4 for 2.00
D#4 for 2.00
D4 fo

In [374]:
import sounddevice as sd

def generate_tone(frequency, duration, sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    # sin with a few overtones
    tone = 0.5 * np.sin(2 * np.pi * frequency * t) + .2 * np.sin(2 * np.pi * 2 * frequency * t) + .07 * np.sin(2 * np.pi * 3 * frequency * t) + .03 * np.sin(2 * np.pi * 4 * frequency * t)
    return tone

def play_tone(frequency, duration):
    audio = generate_tone(frequency, duration)
    sd.play(audio, samplerate=44100)
    sd.wait()  # Wait until the sound has finished playing

def wait(duration):
    sd.play(np.zeros(int(duration * 44100)), samplerate=44100)
    sd.wait()

bpm = 120
time_multiplier = 60 / bpm

# play the sequence
for note in total_seq:
    pitch, dur = decodings[np.argmax(note)]
    adjusted_dur = float(dur) * time_multiplier
    if (pitch <= 0):
        wait(adjusted_dur)
    else:
        play_tone(440 * 2 ** ((pitch - 10) / 12), adjusted_dur)
